In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
data = [   
    1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980
]
y_vec = [   
    76_212_168,
    92_228_496,
    106_021_537,
    123_202_624,
    132_164_569,
    151_325_798,
    179_323_175,
    203_302_031,
    226_542_199,
]

In [7]:
matrix1 = np.array([data[i]**j for i in range(len(data)) for j in range(len(data))], dtype=float).reshape(len(data), len(data))
matrix2 = np.array([(data[i] - 1900)**j for i in range(len(data)) for j in range(len(data))],  dtype=float).reshape(len(data), len(data))
matrix3 = np.array([(data[i] - 1940)**j for i in range(len(data)) for j in range(len(data))], dtype=float).reshape(len(data), len(data))
matrix4 = np.array([((data[i] - 1940) / 40)**j for i in range(len(data)) for j in range(len(data))], dtype=float).reshape(len(data), len(data))

In [9]:
cond1 = np.linalg.cond(matrix1)
cond2 = np.linalg.cond(matrix2)
cond3 = np.linalg.cond(matrix3)
cond4 = np.linalg.cond(matrix4)
print(f"Cond1: {cond1}")
print(f"Cond2: {cond2}")
print(f"Cond3: {cond3}")
print(f"Cond4: {cond4}")

Cond1: 1.434489710526478e+37
Cond2: 6313911222583060.0
Cond3: 9315536039705.943
Cond4: 1605.443700478641


In [11]:
a_vec = np.linalg.solve(matrix4, y_vec)
print(f"a_vec: {a_vec}")

a_vec: [ 1.32164569e+08  4.61307656e+07  1.02716315e+08  1.82527130e+08
 -3.74614715e+08 -3.42668456e+08  6.06291250e+08  1.89175576e+08
 -3.15180235e+08]
